### Bronze Layer structure generation

The default settings in adb lakeflow connect doesnt allow to create table alias and keep the origin schema structure using only 1 pipeline. 

Due to the low-consumption aproach for this PoC. We decided to keep only 1 ingesiton pipeline and use a consolidated schema named stg_all_schemas. Then a LDP job will recreate the structure and feed the corresponding stg schemas.}

This notebook governs the stg schemas table creation and schema structures


In [0]:
import dlt
from pyspark.sql.functions import *

# 1. Configuration (from your image)
source_catalog_schema = "dev_bronze.stg_allschemas"
target_catalog_schema = "dev_bronze.stg_sales"
sales_tables = ["currency", "customer", "salesorderheader", "salesorderdetail"]

# 2. Define a function generator
def create_stg_table(source_name, target_name):
    @dlt.table(
        name=target_name,
        comment=f"Staging table for {source_name}"
    )
    def table_definition():
        # This acts as your 'CREATE OR REPLACE'
        # Use limit(0) if you still want the empty schema only (WHERE 1=0)
        return spark.read.table(source_name).limit(0)

# 3. Iterate to register the tables in the pipeline
for table_name in sales_tables:
    full_source = f"{source_catalog_schema}.{table_name}"
    full_target = f"stg_{table_name}" # DLT usually manages the schema prefix
    
    create_stg_table(full_source, full_target)